# Part 2: Basic LLM Chat Tool

## Introduction

In this part, you'll create a simple command-line chat tool that interacts with a Large Language Model (LLM) through the Hugging Face API. This tool will allow you to have conversations with an LLM about healthcare topics.

## Learning Objectives

- Connect to the Hugging Face API
- Create a basic interactive chat loop
- Handle simple error cases
- Test with healthcare questions

## Setup and Installation

In [1]:
# Install required packages
%pip install -r requirements.txt

# Additional packages for LLM API interaction
%pip install requests

# Import necessary libraries
import os
import sys
import requests
import time
import logging
import argparse
from typing import Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create directories
os.makedirs('utils', exist_ok=True)
os.makedirs('results/part_2', exist_ok=True)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Connecting to the Hugging Face API

The Hugging Face Inference API provides access to many language models. We'll use models that are available on the free tier.

In [7]:
import requests

# Example of a simple API request to Hugging Face
API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-base"
headers = {"Authorization": f"Bearer YOUR_API_KEY"}  # Replace with your actual API key

def query(payload):
    """
    Send a query to the Hugging Face API
    
    Args:
        payload (dict): Dictionary containing the query parameters (e.g., {"inputs": "Your prompt"})
        
    Returns:
        str: The model's response text or error message
    """
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()

        # Some models return a list of outputs with "generated_text"
        if isinstance(result, list) and 'generated_text' in result[0]:
            return result[0]['generated_text']
        elif 'error' in result:
            return f"[Error] {result['error']}"
        else:
            return result
    except requests.exceptions.RequestException as e:
        return f"[Request Failed] {e}"

# Test the query function
test_payload = {"inputs": "What are the symptoms of diabetes?"}
response = query(test_payload)
print(response)

[Request Failed] 401 Client Error: Unauthorized for url: https://api-inference.huggingface.co/models/google/flan-t5-base


## 2. Creating Simple Chat Scripts

Your task is to create two simple scripts that interact with the Hugging Face API:

1. A basic one-off chat script (`utils/one_off_chat.py`)
2. A contextual conversation script (`utils/conversation.py`)

### One-Off Chat Script

Create a script that handles independent interactions (each prompt/response is separate):

In [8]:
import requests
import argparse
import os

def get_response(prompt, model_name="google/flan-t5-base", api_key=None):
    """
    Get a response from the model via Hugging Face API.

    Args:
        prompt (str): User input prompt.
        model_name (str): Name of the HF model to use.
        api_key (str): Hugging Face API key (optional for public models).

    Returns:
        str: Model's response or error message.
    """
    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {api_key}"} if api_key else {}

    payload = {"inputs": prompt}

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()

        if isinstance(result, list) and 'generated_text' in result[0]:
            return result[0]['generated_text']
        elif 'error' in result:
            return f"[Error] {result['error']}"
        else:
            return str(result)

    except requests.exceptions.RequestException as e:
        return f"[Request Failed] {e}"

def run_chat(model_name="google/flan-t5-base", api_key=None):
    """Run an interactive chat session."""
    print("Welcome to the Simple LLM Chat! Type 'exit' to quit.")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        response = get_response(user_input, model_name=model_name, api_key=api_key)
        print("Bot:", response)

def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM (Hugging Face API)")
    parser.add_argument('--model', type=str, default="google/flan-t5-base", help="Model name to use")
    parser.add_argument('--api_key', type=str, default=os.getenv("HF_API_KEY"), help="Hugging Face API Key")

    args = parser.parse_args()

    run_chat(model_name=args.model, api_key=args.api_key)

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--model MODEL] [--api_key API_KEY]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/codespace/.local/share/jupyter/runtime/kernel-v33af537a92e8945c5cfdd930b3fbc0a2575895d62.json


SystemExit: 2

### Contextual Conversation Script

Create a script that maintains conversation history:

In [5]:
import requests
import argparse
import os

def get_response(prompt, history=None, model_name="google/flan-t5-base", api_key=None, history_length=3):
    """
    Get a response from the model using conversation history.

    Args:
        prompt (str): The current user prompt
        history (list): List of previous (user, bot) message pairs
        model_name (str): Hugging Face model name
        api_key (str): Hugging Face API key
        history_length (int): Number of previous turns to include

    Returns:
        str: Model's response
    """
    if history is None:
        history = []

    # Build context with the last `history_length` turns
    context = ""
    for user_input, bot_response in history[-history_length:]:
        context += f"User: {user_input}\nBot: {bot_response}\n"

    # Add current user message
    full_prompt = context + f"User: {prompt}\nBot:"

    # Make API request
    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {api_key}"} if api_key else {}

    payload = {"inputs": full_prompt}

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()

        if isinstance(result, list) and 'generated_text' in result[0]:
            return result[0]['generated_text'].split("Bot:")[-1].strip()
        elif 'error' in result:
            return f"[Error] {result['error']}"
        else:
            return str(result)

    except requests.exceptions.RequestException as e:
        return f"[Request Failed] {e}"

def run_chat(model_name="google/flan-t5-base", api_key=None, history_length=3):
    """Run an interactive contextual chat session."""
    print("Welcome to the Contextual LLM Chat! Type 'exit' to quit.")

    history = []

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        response = get_response(
            prompt=user_input,
            history=history,
            model_name=model_name,
            api_key=api_key,
            history_length=history_length
        )

        print("Bot:", response)

        # Add to conversation history
        history.append((user_input, response))

def main():
    parser = argparse.ArgumentParser(description="Contextual Chat with an LLM")
    parser.add_argument('--model', type=str, default="google/flan-t5-base", help="Model name")
    parser.add_argument('--api_key', type=str, default=os.getenv("HF_API_KEY"), help="Hugging Face API key")
    parser.add_argument('--history_length', type=int, default=3, help="Number of turns of history to keep")

    args = parser.parse_args()

    run_chat(model_name=args.model, api_key=args.api_key, history_length=args.history_length)

if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--model MODEL] [--api_key API_KEY]
                             [--history_length HISTORY_LENGTH]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/codespace/.local/share/jupyter/runtime/kernel-v33af537a92e8945c5cfdd930b3fbc0a2575895d62.json


SystemExit: 2

## 3. Testing and Evaluation

Create a script to test your chat implementations with specific healthcare questions.

In [6]:
# utils/test_chat.py

import os
import csv
from pathlib import Path

# Import our chat modules - since we're in the same directory
from one_off_chat import get_response as get_one_off_response
# Optionally import the conversation module if testing that too
# from conversation import get_response as get_contextual_response

def test_chat(questions, model_name="google/flan-t5-base", api_key=None):
    """
    Test the chat function with a list of questions
    
    Args:
        questions: A list of questions to test
        model_name: Name of the model to use
        api_key: API key for authentication
        
    Returns:
        A dictionary mapping questions to responses
    """
    results = {}
    
    for question in questions:
        print(f"Testing question: {question}")
        # Get response using the one-off chat function
        response = get_one_off_response(question, model_name, api_key)
        results[question] = response
        
    return results

# List of healthcare questions to test
test_questions = [
    "What are the symptoms of gout?",
    "How is gout diagnosed?",
    "What treatments are available for gout?",
    "What lifestyle changes can help manage gout?",
    "What foods should be avoided with gout?"
]

def save_results(results, output_file="results/part_2/example.txt"):
    """
    Save the test results to a file
    
    Args:
        results: Dictionary mapping questions to responses
        output_file: Path to the output file
    """
    with open(output_file, 'w') as f:
        # Write header
        f.write("# LLM Chat Tool Test Results\n\n")
        
        # Write usage examples
        f.write("## Usage Examples\n\n")
        f.write("```bash\n")
        f.write("# Run the one-off chat\n")
        f.write("python utils/one_off_chat.py\n\n")
        f.write("# Run the contextual chat\n")
        f.write("python utils/conversation.py\n")
        f.write("```\n\n")
        
        # Write test results
        f.write("## Test Results\n\n")
        f.write("```csv\n")
        f.write("question,response\n")
        
        for question, response in results.items():
            # Format the question and response for CSV
            q = question.replace(',', '').replace('\n', ' ')
            r = response.replace(',', '').replace('\n', ' ')
            f.write(f"{q},{r}\n")
            
        f.write("```\n")

# Run the test and save results
if __name__ == "__main__":
    results = test_chat(test_questions)
    save_results(results)
    print("Test results saved to results/part_2/example.txt")

ModuleNotFoundError: No module named 'one_off_chat'